# Basic simulation of electrodes in ESPResSo part I:
# Ion-pair in a narrow metallic slit-like confinement using ICC $\star$

In this tutorial we are going to investigate the interaction between ions in
metallic slit pores using **ESPResSo**. 

To work with this tutorial, you should be familiar with the following topics:

1. Setting up and running simulations in ESPResSo - creating particles,
   incorporating interactions.
   If you are unfamiliar with this, you can go through the respective tutorial
   in the `lennard_jones` folder.
2. Basic knowledge of classical electrostatics:
   Dipoles, surface and image charges
3. Reduced units, as described in the ESPResSo user guide
   https://espressomd.github.io/doc/introduction.html#on-units

## Introduction

This tutorial introduces some basic concepts for simulating charges close to an
electrode using **ESPResSo**.
In this first part, we focus on the interaction of a single ion pair confined in
a narrow metallic slit pore using the ELC-IC framework.
Here, we verify the strong deviation from a Coulomb-like interaction using ELC and the analytical solution.
In metallic confinement, the ion pair interaction energy is screened
exponentially due to the presence of induced charges on the slit walls. 
We will also see that the induced charges result in a net attractive force on
the ion as it approaches the wall.

## Theoretical Background 

The normal component of electric field across a surface diving two dielectric
media yields a discontinuity, which can be expressed in terms of a finite
surface charge density 
$(\epsilon_1\vec{E}_1 - \epsilon_2\vec{E}_2).\hat{n}=-\sigma(\vec{r})$.
This expression describes the jump in the electric field across the material
interface going from a dielectric medium $\epsilon_1$ to another one,
$\epsilon_2$.

While in the case of non-polarizable materials ($\epsilon_1 = \epsilon_2 = 1$),
this jump is only related to surface charges and dielectric contrast and the
potential is continuous across the interface, for polarizable materials also the
polarization field $\vec{P}$ will give a contribution. 
In order to solve the problem in presence of a jump of the dielectric constant
across an interface, one must know the electric fields on both sides. 

Another approach is to replace this two domain problem by an equivalent one
without the explicit presence of the dielectric jump.
This is achieved by introducing an additional fictitious charge i.e. an induced
charge density $\sigma_{ind}$ on the surface. 
With this well known "method of image charges", it is sufficient to know the
electric field on one side of the interface. 
This method of an induced charge density at dielectric 2D interface is employed
in ELC-IC to account for the dielectric contrast. 

Partially periodic ionic systems with dielectric interfaces are very often
encountered in the study of energy materials or bio-macromolecular and membrane
studies. 
These systems usually exhibit a confinement along one ($z$) direction, where the confining boundary or interface imposes a dielectric discontinuity, while the other $x$-$y$ directions are periodic and exhibit bulk-like properties.

The "electrostatic layer correction” (ELC) method proposed by Tyagi et al.
<a href='#[1]'>[1]</a> accounts for the periodic contributions in the constrained
direction by evaluating a layer correction term growing linear with the particle
number $N$, which then can be used with any, efficiently scaling 3-$D$ Coulomb
solvers. 
We here use P3M (particle-particle, particle-mesh), which scales with 
$\mathcal{O}(N \log N)$.
Furthermore, ELC can be extended to implement the dielectric interfaces by using
the method of image charges and this is the "Electrostatic layer correction with
image charges" (ELC-IC) approach used in this tutorial.
  
*Note*: Apart from ELC-IC, **ESPResSo** also provides ICC method where we account
for a set of spatially fixed ICC particles, whose initial charges are iterated
over time until they correctly represent the influence of the dielectric
discontinuity.

First we import all ESPResSo features and external modules

In [ ]:
import espressomd
import numpy as np
import espressomd.electrostatics
import espressomd.electrostatic_extensions
from espressomd.interactions import *
import espressomd.visualization
import espressomd.observables
import espressomd.accumulators
import matplotlib.pyplot as plt
from scipy.special import *


from tqdm import tqdm

##  System setup 

We define system dimensions and some physical parameters related to length, time
and energy scales of our system.
All physical parameters are defined in reduced units of length ($\sigma=1$;
Particle size), mass ($m=1$; Particle mass), time ($t=0.01 \tau$) and
elementary charge ($e=1$).

Another important length scale is the Bjerrum Length, which is the length at
which the electrostatic energy between two elementary charges is comparable to
the thermal energy $k_\mathrm{B}T$.
It is defined as $l_\mathrm{B}=\frac{1}{4\pi\epsilon_0\epsilon_rk_BT}$. 
In our case if we choose the ion size ($\sigma$) in simulations equivalent to a
typical experimental value for mono-atomar salt, 0.3 nm in real units, then the
Bjerrum length of water at room temperature, $\ell_\mathrm{B}=0.71 \,\mathrm{nm}$ is
$\ell_\mathrm{B}\sim 2$ in simulations units.

In [ ]:
#***************************************************
#            System Setup
#***************************************************

# Box dimensions
# To construct a narrow slit Lz << (Lx , Ly)
box_l_x = 100.
box_l_y = 100.
box_l_z = 5.

# ICC* with vacuum layer
VACUUM_GAP = 6.0*box_l_z

system = espressomd.System(box_l=[box_l_x, box_l_y, box_l_z+VACUUM_GAP])

# System Time
system.time_step = 0.01
system.cell_system.skin = 0.4

# Elementary charge 
q = np.array([1.0])  

# Interaction Parameters: P3M with ELCIC

BJERRUM_LENGTH = 2.0        # Electrostatic prefactor passed to P3M ; prefactor=lB KBT/e2                
ACCURACY = 1e-6             # P3M force accuracy      

#Lennard-Jones  Parameters

LJ_SIGMA = 1.0
LJ_EPSILON = 1.0                

#Particle parameters

types = {"Cation": 0, "Anion": 1  ,"Electrodes": 2}
charges = {"Cation": q[0], "Anion": -q[0]  }

p1=system.part.add(pos=[box_l_x/2.0, box_l_y/2.0,0.0], q=charges["Cation"])
print(f"Cation placed at position: {p1.pos}")
p2=system.part.add(pos=p1.pos + [0, 0, box_l_z/2.0],q=charges["Anion"])
print(f"Anion placed at position: {p2.pos}")


In [ ]:
p3m = espressomd.electrostatics.P3M(
            prefactor=BJERRUM_LENGTH,
            accuracy=ACCURACY,
            check_neutrality = False,
            mesh = [50,50,70],
            cao = 5
        )

# Set the ICC line density and calculate the number of
# ICC particles according to the box size
nicc = 100  # linear density
nicc_per_electrode = nicc**2  # surface density
nicc_tot = 2 * nicc_per_electrode
iccArea = box_l_x * box_l_y / nicc_per_electrode
lx = box_l_x / nicc
ly = box_l_y / nicc

# Lists to collect required parameters
iccNormals = []
iccAreas = []
iccSigmas = []
iccEpsilons = []

# Add the fixed ICC particles:

# Bottom electrode (normal [0, 0, 1])
for xi in tqdm(range(nicc)):
    for yi in range(nicc):
        system.part.add(pos=[lx * xi, ly * yi, 0.], q=-0.0001,
                        type=types["Electrodes"], fix=[True, True, True])
iccNormals.extend([0, 0, 1] * nicc_per_electrode)

# Top electrode (normal [0, 0, -1])
for xi in tqdm(range(nicc)):
    for yi in range(nicc):
        system.part.add(pos=[lx * xi, ly * yi, box_l_z], q=0.0001,
                        type=types["Electrodes"], fix=[True, True, True])
iccNormals.extend([0, 0, -1] * nicc_per_electrode)

# Common area, sigma and metallic epsilon
iccAreas.extend([iccArea] * nicc_tot)
iccSigmas.extend([0] * nicc_tot)
iccEpsilons.extend([100000] * nicc_tot)

iccNormals = np.array(iccNormals, dtype=float).reshape(nicc_tot, 3)

icc = espressomd.electrostatic_extensions.ICC(
    first_id=2,
    n_icc=nicc_tot,
    convergence=1e-5,
    relaxation=0.75,
    ext_field=[0, 0, 0],
    max_iterations=200,
    eps_out=1.0,
    normals=iccNormals,
    areas=np.array(iccAreas, dtype=float),
    sigmas=np.array(iccSigmas, dtype=float),
    epsilons=np.array(iccEpsilons, dtype=float)
    )

system.electrostatics.solver = p3m
system.electrostatics.extension = icc

In [ ]:
#******************CASE 3 *****************************************
#   Interaction energy of a dipole confined between dielectric walls 
#   as a function of separation R  
# *****************************************************************

R = np.linspace(0.5, box_l_x / 4.0  ,10)
#R = np.array([5])
elc_forces_axial = np.empty((len(R), 2))
elc_energy_axial = np.empty(len(R))

for i, x in enumerate(tqdm(R)):
    p1.pos = [0, box_l_y/2.0, box_l_z/2.0]
    p2.pos = [x, box_l_y/2.0, box_l_z/2.0]
    
    system.integrator.run(0)
    elc_forces_axial[i, 0] = p1.f[0]
    elc_forces_axial[i, 1] = p2.f[0]
    elc_energy_axial[i] = system.analysis.energy()[
                    "total"]

In [ ]:
#x = np.linspace(const.physical_constants['Bohr radius'][0]/const.angstrom,50)
x = np.linspace(0.5,box_l_x/4.0,100)

def slavko(R, w):
    prefactor = 4/ w

    E = 0; n=int(1e5)
    for i in np.arange(1, n+1):
        E += k0(np.pi * i *  R / w) 
    E *= prefactor
    return E

def loche(R,w):
    x = R/w
    prefactor = 2 / (w)

    def summand(x, n):
        return (-1)**n/np.sqrt(x**2+n**2)
    
    def bla(x):
        max = int(1e4)
        sum = 0
        for n in range(-max+1,max+1):
            sum += summand(x,n)
        return sum

    E = bla(x) * prefactor
    return E

def coulomb(x):
    prefactor = 1
    E = prefactor / x
    return E

#for i,L in enumerate([box_l_z]):
#    #plt.plot(x, slavko(x,L), ls='-', color='b', label=['Kondrat Chem. Rev. 2023 Eq. (9)',''][i!=0])
#    plt.plot(x, loche(x,L), ls=':', color='r', label='Loche JPCB 2020 Eq. (S21)', lw=4)
#    plt.plot(x, coulomb(x), ls='-.', color='k', label='Coulomb')
#
#plt.plot(R, -elc_energy_axial,  color='red' , label="sim",  marker='o')
#
#plt.loglog()
#plt.legend()
#plt.xlabel(r'$R\,\mathrm{[\AA]}$')
#plt.ylabel(r'$E\,\mathrm{[Hartree]}$')

In [ ]:
for i,L in enumerate([box_l_z]):
    #plt.plot(x, -np.gradient(slavko(x,L),x), ls='-', color='b', label=['Kondrat Chem. Rev. 2023 Eq. (9)',''][i!=0])
    plt.plot(x, -np.gradient(loche(x,L), x), ls=':', color='b', label='Loche JPCB 2020 Eq. (S21)', lw=4)
    plt.plot(x, -np.gradient(coulomb(x), x), ls='-.', color='k', label='Coulomb')

plt.plot(R, -elc_forces_axial[:,1],  color='red' , label="sim",  marker='o')
plt.loglog()
plt.legend()

## References

<a id='[1]'></a>[1] Tyagi, S.; Arnold, A.; Holm, C. Electrostatic Layer Correction with Image Charges: A Linear Scaling Method to Treat Slab 2D+h Systems with Dielectric Interfaces. J. Chem. Phys. 2008, 129 (20), 204102. https://doi.org/10.1063/1.3021064.
